In [1]:
# @title
!pip install --upgrade pip --quiet
!pip install galpy ipywidgets --quiet

# Import libraries
import numpy as np
import matplotlib.pyplot as plt
from galpy.potential import MiyamotoNagaiPotential, NFWPotential, calcRotcurve
from astropy import units
from ipywidgets import interactive, FloatSlider, Checkbox, Label, HBox, VBox, HTML
from IPython.display import display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.1 MB/s eta 0:00:00


In [2]:
# @title
# Parameters
r_0 = 1     # kpc
v_0 = 220   # km/s

vmaxplot = 300

# Bulge parameters
amp1 = 1.06E+10
a1 = 0
b1 = 0.3
threshold_mass1 = 1
threshold_a1 = 20
threshold_b1 = 70

# Thin disk parameters
amp2 = 3.94E+10
a2 = 5.3
b2 = 0.25
threshold_mass2 = 1
threshold_a2 = 90
threshold_b2 = 1

# Dark halo parameters
amp3 = 1.39E+11
a3 = 14
threshold_mass3 = 1
threshold_a3 = 90

# Data to use as reference
r_data = np.array([6.1, 6.2, 6.3, 6.4, 6.5, 6.6, 6.7, 6.8, 6.9, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5, 7.6, 7.7, 7.8, 7.9, 8.0, 8.1, 8.2, 8.3, 8.4, 8.5, 8.6, 8.7, 8.8, 8.9, 9.0, 9.1, 9.2, 9.3, 9.4, 9.5, 9.6, 9.7, 9.8, 9.9, 10.0, 10.1, 10.2, 10.3, 10.4])
v_c_data = np.array([259.00100624, 245.83008077, 264.13648452, 252.35407172, 257.63036007, 253.83088384, 252.63485120, 253.42512028, 254.57925507, 252.42174922, 251.24248088, 252.79911954, 252.49922645, 250.96057091, 251.65353027, 253.44650486, 251.43197811, 253.02512027, 252.31313062, 251.92387732, 250.96505738, 249.74295440, 249.78403967, 249.49309041, 248.99815855, 248.14590788, 248.29915218, 247.28022074, 244.56514048, 246.51556527, 246.95394532, 242.63435566, 241.01047447, 239.91593826, 241.19068707, 236.54324295, 236.73355387, 241.35980413, 235.19375562, 235.70322181, 229.42882313, 229.72795324, 229.75794386, 235.89062626])
v_c_err_data = np.array([4.55994866, 3.61827523, 3.60403704, 2.33929631, 2.15218155, 1.75066642, 1.65992982, 1.43755106, 1.15522503, 1.09603464, 0.86639895, 0.75280676, 0.62289050, 0.52816485, 0.44368477, 0.39314470, 0.36075107, 0.28098164, 0.22980847, 0.17843819, 0.14645920, 0.12313338, 0.12369428, 0.17868982, 0.23919664, 0.30151008, 0.36607491, 0.42218066, 0.50033514, 0.53778709, 0.70422332, 0.82776641, 0.88668507, 1.18393589, 1.20648711, 1.37857619, 1.86373994, 1.93277101, 2.04128754, 2.59433701, 4.55446709, 2.58715820, 4.43849895, 3.71093587])

# Define bins for the plot
r_bins = np.linspace(0.00001, np.max(r_data) * 1.01, 10*len(r_data))

In [3]:
# @title
def mn_bulge_rotcurve(amp1, a1, b1):
    MN_Bulge_p = MiyamotoNagaiPotential(
        amp = amp1 * 1e9 * units.Msun,
        a = a1 * units.kpc,
        b = b1 * units.kpc,
        normalize = False,
        ro=r_0 * units.kpc,
        vo=v_0 * units.km/units.s
    )
    MN_Bulge = calcRotcurve(MN_Bulge_p, r_bins, phi=None) * v_0
    return MN_Bulge, MN_Bulge_p

def mn_thin_disk_rotcurve(amp2, a2, b2):
    MN_Thin_Disk_p = MiyamotoNagaiPotential(
        amp = amp2 * 1e9 * units.Msun,
        a = a2 * units.kpc,
        b = b2 * units.kpc,
        normalize = False,
        ro=r_0 * units.kpc,
        vo=v_0 * units.km/units.s
    )
    MN_Thin_Disk = calcRotcurve(MN_Thin_Disk_p, r_bins, phi=None) * v_0
    return MN_Thin_Disk, MN_Thin_Disk_p

def nfw_rotcurve(amp3, a3):
    NFW_p = NFWPotential(
        amp = amp3 * 1e9 * units.Msun,
        a = a3 * units.kpc,
        normalize = False,
        ro=r_0 * units.kpc,
        vo=v_0 * units.km/units.s
    )
    NFW = calcRotcurve(NFW_p, r_bins, phi=None) * v_0
    return NFW, NFW_p

def plot_combined(amp1, a1, b1, amp2, a2, b2, amp3, a3):
    # Calculate the rotation curve for each potential
    MN_Bulge, MN_Bulge_p = mn_bulge_rotcurve(amp1, a1, b1)
    MN_Thin_Disk, MN_Thin_Disk_p = mn_thin_disk_rotcurve(amp2, a2, b2)
    NFW, NFW_p = nfw_rotcurve(amp3, a3)
    # Calculate the rotation curve for the combined potentials
    v_circ_comp = calcRotcurve([MN_Bulge_p, MN_Thin_Disk_p, NFW_p], r_bins, phi=None) * v_0
    # Plot the rotation curve
    plt.figure(figsize=(8, 4))
    # Potentials
    plt.plot(r_bins, MN_Bulge, linestyle='--', c='gray')
    plt.plot(r_bins, MN_Thin_Disk, linestyle='--', c='lightblue')
    plt.plot(r_bins, NFW, linestyle='--', c='green')
    plt.plot(r_bins, v_circ_comp, c='k')
    # Reference data
    plt.scatter(r_data, v_c_data, s=3, c='k', label='Data', alpha=0.5)
    plt.errorbar(r_data, v_c_data, v_c_err_data, c='k', fmt='', ls='none')
    # General plot settings
    plt.title('Rotation Curve with theoretical components')
    plt.xlabel(r'$R$ (kpc)')
    plt.ylabel(r'$v_{\phi}$ (km/s)')
    plt.xlim([0, r_data[-1] * 1.01])
    plt.ylim([0, vmaxplot])
    plt.show()

In [4]:
# @title
# Create interactive sliders
# MN Bulge sliders
label1 = Label(value="MN Bulge (GRAY)")
amp1_slider = FloatSlider(value = amp1 / 1e9, min = amp1 / (10**threshold_mass1) / 1e9, max = amp1 * (10**threshold_mass1) / 1e9, step = 1, description='M (10^9 M☉)')
a1_slider = FloatSlider(value = a1, min = 0, max = 0.01 * threshold_a1, step = 0.01, description='a (kpc)')
b1_slider = FloatSlider(value = b1, min = b1 * (1 - 0.01 * threshold_b1), max = b1 * (1 + 0.01 * threshold_b1), step = 0.01, description='b (kpc)')
# MN Thin Disk sliders
label2 = Label(value="MN Thin Disk (BLUE)")
amp2_slider = FloatSlider(value = amp2 / 1e9, min = amp2 / (10**threshold_mass2) / 1e9, max = amp2 * (10**threshold_mass2) / 1e9, step = 1, description='M (10^9 M☉)')
a2_slider = FloatSlider(value = a2, min = a2 * (1 - 0.01 * threshold_a2), max = a2 * (1 + 0.01 * threshold_a2), step = 0.01, description='a (kpc)')
b2_slider = FloatSlider(value = b2, min = b2 / (10**threshold_b2), max = b2 * (10**threshold_b2), step = 0.01, description='b (kpc)')
# NFW sliders
label3 = Label(value="NFW - Halo (GREEN)")
amp3_slider = FloatSlider(value = amp3 / 1e9, min = amp3 / (10**threshold_mass3) / 1e9, max = amp3 * (10**threshold_mass3) / 1e9, step = 1, description='M (10^9 M☉)')
a3_slider = FloatSlider(value = a3, min = a3 * (1 - 0.01 * threshold_a3), max = a3 * (1 + 0.01 * threshold_a3), step = 0.01, description='a (kpc)')

# Display the interactive plot
interactive_plot = interactive(plot_combined, amp1=amp1_slider, a1=a1_slider, b1=b1_slider, amp2=amp2_slider, a2=a2_slider, b2=b2_slider, amp3=amp3_slider, a3=a3_slider)
interactive_plot_output = interactive_plot.children[-1]
controls = VBox([
    VBox([label1, amp1_slider, a1_slider, b1_slider]),
    VBox([label2, amp2_slider, a2_slider, b2_slider]),
    VBox([label3, amp3_slider, a3_slider])
])
display(HBox([controls, interactive_plot_output]))